## QLearning简单的实现

利用探索者从左到右去寻找最右端的宝藏<br>
* ACTIONS：向左或者向右<br>
* N_STATES：从最左端0到最右端terminate-6，共6个状态<br>
* Reward 代表在状态s做出动作a时给予的奖励（若s=5时向右为1，其他均为0）

### 具体算法
初始化Q(s,a)表格，其中s是状态，a是动作<br>
Repeat (for each eposode):<br>
&#160;&#160;&#160;&#160; 初始化 s<br>
&#160;&#160;&#160;&#160; Repeat (for each step of eposode):<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; 从Q表格中根据当前状态s选择动作a（在$\epsilon = 0.9$ 的概率下，否则随机选择动作）<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; 根据动作a，得到当前的观察r和状态s'<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; $Q(s,a) = Q(s,a) + \alpha [r + \gamma max_{a'}Q(s',a')-Q(s,a)]$ 这里a表示动作、s表示状态；s'和a'表示下一个状态和动作<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; s = s'<br>
&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160; 直到s是终点，结束循环<br>


In [1]:
import numpy as np
import pandas as pd
import time

np.random.seed(2)  # reproducible

N_STATES = 6   # the length of the 1 dimensional world
ACTIONS = ['left', 'right']     # available actions
EPSILON = 0.9   # greedy police
ALPHA = 0.1     # learning rate
GAMMA = 0.9    # discount factor
MAX_EPISODES = 13   # maximum episodes
FRESH_TIME = 0.3    # fresh time for one move

### 建立空的Q表格

In [2]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table initial values
        columns=actions,    # actions's name
    )
    # print(table)    # show table
    return table

build_q_table(N_STATES,ACTIONS)

,left,right
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0


### 根据当前状态和Q表格选择下一步的动作
假设在状态0时候向左的reward是-1，向右的是0

In [3]:
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(ACTIONS)
    else:   # act greedy
        action_name = state_actions.idxmax()    # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name

q_table = build_q_table(N_STATES,ACTIONS)
q_table['left'][0] = -1
choose_action(0, q_table)

'right'

### 更新状态，得到下一步的状态$s'$和奖赏Reward
如果在terminate的前面一步选择向右，则R=1，否则都为0；如果在状态0的时候继续向左，默认继续为状态0

In [14]:
def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R

print(get_env_feedback(3, 'right'))
print(get_env_feedback(4, 'right'))

(4, 0)
('terminal', 1)


### 更新环境

利用--o--T的样式来记录环境。初始为---------T，o代表当前的状态，T代表teminate

In [7]:
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        print('\rEpisode %s: total_steps = %s' % (episode+1, step_counter))
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)
        
update_env(0, 0.9, 0)
update_env(1, 0.9, 1)
update_env(2, 0.9, 2)

--o--T

### 实现QLearning的主干部分
$Q(s,a) = Q(s,a) + \alpha [r + \gamma max_{a'}Q(s',a')-Q(s,a)]$<br>

$Q(s',a')$即q_target，$Q(s,a)$即q_predict，$r$为reward，$\gamma$ 为衰减系数，$\alpha$ 为学习率

In [21]:
def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:

            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)  # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state

            update_env(S, episode, step_counter+1)
            step_counter += 1
        print('\r\nQ-table:\n')
        print(q_table)
        print('\r')
    return q_table

In [22]:
q_table = rl()

Episode 1: total_steps = 31
                                
Q-table:

   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.1
5   0.0    0.0

Episode 2: total_steps = 10
                                
Q-table:

   left  right
0   0.0  0.000
1   0.0  0.000
2   0.0  0.000
3   0.0  0.009
4   0.0  0.190
5   0.0  0.000

Episode 3: total_steps = 19
                                
Q-table:

   left    right
0   0.0  0.00000
1   0.0  0.00000
2   0.0  0.00081
3   0.0  0.02520
4   0.0  0.27100
5   0.0  0.00000

Episode 4: total_steps = 14
                                
Q-table:

   left     right
0   0.0  0.000000
1   0.0  0.000073
2   0.0  0.002997
3   0.0  0.047070
4   0.0  0.343900
5   0.0  0.000000

Episode 5: total_steps = 6
                                
Q-table:

   left     right
0   0.0  0.000007
1   0.0  0.000335
2   0.0  0.006934
3   0.0  0.073314
4   0.0  0.409510
5   0.0  0.000000

Episode 6: total_steps = 5
                                
